# NWA suntans: make movie

In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr

%matplotlib inline
from matplotlib import pyplot as plt

import pynsitu as pin
crs = pin.maps.crs

import nwatools as nwa

ERROR 1: PROJ: proj_create_from_database: Open of /home1/datahome/aponte/.miniconda3/envs/equinox/share/proj failed


In [2]:
if True:
    from dask.distributed import Client, LocalCluster
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster()
    #cluster = PBSCluster(cores=28, processes=14)
    #cluster = PBSCluster(cores=14, processes=7)
    w = cluster.scale(jobs=10)
    #
    client = Client(cluster)
    client
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()
    client = Client(cluster)
    client

---
## load data

In [3]:
ds, grd = nwa.load_surf()

In [4]:
# add deformations
zarr = os.path.join(nwa.suntans_dir, f"suntans_2km_surf_gradients")
dsd = xr.open_zarr(zarr)

dsd["vorticity"] = (dsd["dvcdx"] - dsd["ducdy"])/pin.geo.coriolis(dsd.yv)

#dsd["divergence"] = dsd["ducdx"] + dsd["dvcdy"]
#dsd["strain_normal"] = dsd["ducdx"] - dsd["dvcdy"]
#dsd["strain_shear"] = dsd["dvcdx"] + dsd["ducdy"]

ds = dsd[["vorticity"]]

In [5]:
ds = ds.persist()

In [6]:
# movie parameters
v = "vorticity"
cmap = 'RdBu_r'

#clims = (-1,1)
#zoom = "large"

clims = (-2,2)
zoom = "cp"

#time = dsd["time"][:100]
time = dsd["time"]

fig_dir = "/home1/scratch/aponte/figs"

In [7]:
from dask import delayed
import threading
from tqdm import tqdm

In [8]:
def print_figure(da, i):
    
    MPL_LOCK = threading.Lock()
    with MPL_LOCK:
        
        plt.switch_backend('agg')
        
        fig, ax = nwa.map_init(zoom, bathy=(grd, da))
        _, _, poly, cbar = grd.suntans.plotcelldata(da, 
                                                    vmin=clims[0], vmax=clims[1], 
                                                    cmap=cmap, 
                                                    crs=crs,
                                                   )

        t = str(da.time.dt.strftime('%Y/%m/%d %Hh').values)
        ax.set_title(t)
        
        fig_file = os.path.join(fig_dir, f"{i:05d}.png")
        fig.savefig(fig_file, dpi=150)
        plt.close(fig)
        
print_figure_delayed = delayed(print_figure)

In [9]:
n_t = ds.time.size
n_workers = len(client.scheduler_info()['workers'])

for i in tqdm(range(0, n_t, n_workers)):
    values = [print_figure_delayed(ds[v].isel(time=j), j) 
              for j in range(i, min(i+n_workers, n_t))]    
    futures = client.compute(values)
    results = client.gather(futures)

100%|██████████| 126/126 [1:33:03<00:00, 44.31s/it]


---

In [ ]:
cluster.close()